## Análise de Sentimento em Avaliações de Usuários

In [1]:
# Imports
import re
import pickle
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score

In [2]:
# Carrega o dataset
dados = pd.read_csv('dataset.csv')

In [3]:
# Corpo
dados.shape

(50000, 2)

In [4]:
# Amostra dos dados
dados.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
# Info
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
# Contagem de registros por classe
dados.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

## Limpeza dos Dados

In [7]:
# Ajustamos os labels para representação numérica
dados.sentiment.replace('positive', 1, inplace = True)
dados.sentiment.replace('negative', 0, inplace = True)

/tmp/ipykernel_13827/3288419048.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados.sentiment.replace('positive', 1, inplace = True)
/tmp/ipykernel_13827/3288419048.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [8]:
# Amostra dos dados
dados.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
# Vamos observar uma avaliação de usuário
dados.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [10]:
# Função de limpeza geral de dados
def limpa_dados(texto):
    cleaned = re.compile(r'<.*?>') 
    return re.sub(cleaned, '', texto)

In [11]:
# Testando a função
texto_com_tags = "<p>Este é um exemplo <b>com</b> tags HTML.</p>"
texto_limpo = limpa_dados(texto_com_tags)
print(texto_limpo) 

Este é um exemplo com tags HTML.


In [12]:
# Aplica a função ao nosso dataset
dados.review = dados.review.apply(limpa_dados)

In [13]:
dados.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [14]:
# Função para limpeza de caracteres especiais
def limpa_caracter_especial(texto):
    rem = ''
    for i in texto:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
            
    return rem

In [15]:
# Testando a função
texto_com_caracteres_especiais = "Olá, mundo! Como vai?"
texto_limpo = limpa_caracter_especial(texto_com_caracteres_especiais)
print(texto_limpo)

Olá  mundo  Como vai 


In [16]:
# Aplica a função
dados.review = dados.review.apply(limpa_caracter_especial)

In [17]:
dados.review[0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked  They are right  as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [18]:
# Função para converter o texto em minúsculo
def converte_minusculo(texto):
    return texto.lower()

In [19]:
# Testando a função
frase = "Esta é uma fraSE com LETRAS MaiúscuLAs"
frase_saida = converte_minusculo(frase)
print(frase_saida)

esta é uma frase com letras maiúsculas


In [20]:
# Aplica a função
dados.review = dados.review .apply(converte_minusculo)

In [21]:
dados.review[0]

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [22]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/everton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/everton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
# Função para remover stopwords
def remove_stopwords(texto):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(str(texto))
    return [w for w in words if w not in stop_words]

In [24]:
 import nltk
 nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/everton/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [25]:
# Testando a função
frase =  "They are right, as this is exactly what happened with me."
frase_saida = remove_stopwords(frase)
print(frase_saida)

['They', 'right', ',', 'exactly', 'happened', '.']


In [26]:
%%time
dados.review = dados.review.apply(remove_stopwords)

CPU times: user 37.6 s, sys: 781 ms, total: 38.4 s
Wall time: 38.4 s


In [27]:
dados.review[0]

['one',
 'reviewers',
 'mentioned',
 'watching',
 '1',
 'oz',
 'episode',
 'hooked',
 'right',
 'exactly',
 'happened',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'right',
 'word',
 'go',
 'trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs',
 'sex',
 'violence',
 'hardcore',
 'classic',
 'use',
 'word',
 'called',
 'oz',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'focuses',
 'mainly',
 'emerald',
 'city',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards',
 'privacy',
 'high',
 'agenda',
 'em',
 'city',
 'home',
 'many',
 'aryans',
 'muslims',
 'gangstas',
 'latinos',
 'christians',
 'italians',
 'irish',
 'scuffles',
 'death',
 'stares',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'away',
 'would',
 'say',
 'main',
 'appeal',
 'show',
 'due',
 'fact',
 'goes',
 'shows',
 'da

In [28]:
# Função para o stemmer
def stemmer(texto):
    objeto_stemmer = SnowballStemmer('english')
    return " ".join([objeto_stemmer.stem(w) for w in texto])

In [29]:
# Testando a função
texto = "The cats are running"
texto_stemmed = stemmer(texto.split())
print(texto_stemmed)  

the cat are run


In [30]:
%%time
dados.review = dados.review.apply(stemmer)

CPU times: user 59.9 s, sys: 26.1 ms, total: 60 s
Wall time: 60 s


In [31]:
dados.review[0]

'one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

## Pré-Processamento dos Dados

In [32]:
# Extrai o texto da avaliação (entrada)
x = np.array(dados.iloc[:,0].values)

In [33]:
# Extrai o sentimento (saída)
y = np.array(dados.sentiment.values)

In [34]:
# Divisão dos dados em treino e teste com proporção 80/20
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.2, random_state = 0)

In [35]:
# Cria um vetorizador (vai converter os dados de texto em representação numérica)
vetorizador = CountVectorizer(max_features = 1000)

In [36]:
# Fit e transform do vetorizador com dados de treino
x_treino_final = vetorizador.fit_transform(x_treino).toarray()

In [37]:
# Apenas transorm nos dados de teste
x_teste_final = vetorizador.transform(x_teste).toarray()

In [38]:
print("x_treino_final:", x_treino_final.shape)
print("y_treino:", y_treino.shape)

x_treino_final: (40000, 1000)
y_treino: (40000,)


In [39]:
print(x_treino_final)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [40]:
print("x_teste_final:", x_teste_final.shape)
print("y_teste:", y_teste.shape)

x_teste_final: (10000, 1000)
y_teste: (10000,)


## Criação de Modelos de Machine Learning

#### Modelo Probabilístico 1 - GaussianNB


In [41]:
# Cria o modelo
modelo_v1 = GaussianNB()

In [42]:
# Treina o modelo
modelo_v1.fit(x_treino_final, y_treino)

GaussianNB()

#### Modelo Probabilístico 2 - MultinomialNB

In [43]:
# Cria o modelo
modelo_v2 = MultinomialNB(alpha = 1.0, fit_prior = True)

In [44]:
# Treina o modelo
modelo_v2.fit(x_treino_final, y_treino)

MultinomialNB()

#### Modelo Probabilístico 3 - BernoulliNB

In [45]:
# Cria o modelo
modelo_v3 = BernoulliNB(alpha = 1.0, fit_prior = True)

In [46]:
# Treina o modelo
modelo_v3.fit(x_treino_final, y_treino)

BernoulliNB()

## Avaliação, Interpretação e Comparação dos Modelos

In [47]:
# Previsões com dados de teste
ypred_v1 = modelo_v1.predict(x_teste_final)

In [48]:
# Previsões com dados de teste
ypred_v2 = modelo_v2.predict(x_teste_final)

In [49]:
# Previsões com dados de teste
ypred_v3 = modelo_v3.predict(x_teste_final)

In [50]:
print("Acurácia do Modelo GaussianNB = ", accuracy_score(y_teste, ypred_v1) * 100)
print("Acurácia do Modelo MultinomialNB = ", accuracy_score(y_teste, ypred_v2) * 100)
print("Acurácia do Modelo BernoulliNB = ", accuracy_score(y_teste, ypred_v3) * 100)

Acurácia do Modelo GaussianNB =  79.06
Acurácia do Modelo MultinomialNB =  82.57
Acurácia do Modelo BernoulliNB =  83.02000000000001


In [51]:
# Import
from sklearn.metrics import roc_auc_score

A acurácia é uma métrica global ideal para comparar versões do modelo do mesmo algoritmo. Para modelos de algoritmos diferentes a métrica AUC (Area Under The Curve) é a ideal.

In [52]:
# AUC do GaussianNB
y_proba = modelo_v1.predict_proba(x_teste_final)[:, 1]
auc = roc_auc_score(y_teste, y_proba)
print("AUC do Modelo GaussianNB =", auc)

AUC do Modelo GaussianNB = 0.861081232980416


In [53]:
# AUC do MultinomialNB
y_proba = modelo_v2.predict_proba(x_teste_final)[:, 1]
auc = roc_auc_score(y_teste, y_proba)
print("AUC do Modelo MultinomialNB =", auc)

AUC do Modelo MultinomialNB = 0.8993217067636314


In [54]:
# AUC do BernoulliNB
y_proba = modelo_v3.predict_proba(x_teste_final)[:, 1]
auc = roc_auc_score(y_teste, y_proba)
print("AUC do Modelo BernoulliNB =", auc)

AUC do Modelo BernoulliNB = 0.9083430688103717


In [55]:
# Salva o melhor modelo em disco
with open('modelo_v3.pkl', 'wb') as arquivo:
    pickle.dump(modelo_v3, arquivo)

## Deploy e Uso do Modelo

In [56]:
# Carrega o modelo do disco
with open('modelo_v3.pkl', 'rb') as arquivo:
    modelo_final = pickle.load(arquivo)

In [57]:
# Texto de uma avaliação de usuário (esse texto apresenta sentimento positivo)
texto_aval = """This is probably the fastest-paced and most action-packed of the German Edgar Wallace ""krimi"" 
series, a cross between the Dr. Mabuse films of yore and 60's pop thrillers like Batman and the Man 
from UNCLE. It reintroduces the outrageous villain from an earlier film who dons a stylish monk's habit and 
breaks the necks of victims with the curl of a deadly whip. Set at a posh girls' school filled with lecherous 
middle-aged professors, and with the cops fondling their hot-to-trot secretaries at every opportunity, it 
certainly is a throwback to those wonderfully politically-incorrect times. There's a definite link to a later 
Wallace-based film, the excellent giallo ""Whatever Happened to Solange?"", which also concerns female students 
being corrupted by (and corrupting?) their elders. Quite appropriate to the monk theme, the master-mind villain 
uses booby-trapped bibles here to deal some of the death blows, and also maintains a reptile-replete dungeon 
to amuse his captive audiences. <br /><br />Alfred Vohrer was always the most playful and visually flamboyant 
of the series directors, and here the lurid colour cinematography is the real star of the show. The Monk appears 
in a raving scarlet cowl and robe, tastefully setting off the lustrous white whip, while appearing against 
purplish-night backgrounds. There's also a voyeur-friendly turquoise swimming pool which looks great both 
as a glowing milieu for the nubile students and as a shadowy backdrop for one of the murder scenes. 
The trademark ""kicker"" of hiding the ""Ende"" card somewhere in the set of the last scene is also quite 
memorable here. And there's a fine brassy and twangy score for retro-music fans.<br /><br />Fans of the series 
will definitely miss the flippant Eddie Arent character in these later films. Instead, the chief inspector 
Sir John takes on the role of buffoon, convinced that he has mastered criminal psychology after taking a few 
night courses. Unfortunately, Klaus Kinski had also gone on to bigger and better things. The krimis had 
lost some of their offbeat subversive charm by this point, and now worked on a much more blatant pop-culture 
level, which will make this one quite accessible to uninitiated viewers."""

In [58]:
# Fluxo de transformação dos dados
tarefa1 = limpa_dados(texto_aval)
tarefa2 = limpa_caracter_especial(tarefa1)
tarefa3 = converte_minusculo(tarefa2)
tarefa4 = remove_stopwords(tarefa3)
tarefa5 = stemmer(tarefa4)

In [59]:
print(tarefa5)

probabl fastest pace action pack german edgar wallac krimi seri cross dr mabus film yore 60 pop thriller like batman man uncl reintroduc outrag villain earlier film don stylish monk habit break neck victim curl dead whip set posh girl school fill lecher middl age professor cop fondl hot trot secretari everi opportun certain throwback wonder polit incorrect time definit link later wallac base film excel giallo whatev happen solang also concern femal student corrupt corrupt elder quit appropri monk theme master mind villain use boobi trap bibl deal death blow also maintain reptil replet dungeon amus captiv audienc alfr vohrer alway play visual flamboy seri director lurid colour cinematographi real star show monk appear rave scarlet cowl robe tast set lustrous white whip appear purplish night background also voyeur friend turquois swim pool look great glow milieu nubil student shadowi backdrop one murder scene trademark kicker hide end card somewher set last scene also quit memor fine bra

In [60]:
type(tarefa5)

str

In [61]:
# Convertendo a string para um array Numpy (pois foi assim que o modelo foi treinado)
tarefa5_array = np.array(tarefa5)

In [62]:
type(tarefa5_array)

numpy.ndarray

In [63]:
# Aplicamos o vetorizador com mais uma conversão para array NumPy a fim de ajustar o shape de 0-d para 1-d
aval_final = vetorizador.transform(np.array([tarefa5_array])).toarray()

In [64]:
type(aval_final)

numpy.ndarray

In [65]:
# Previsão com o modelo
previsao = modelo_final.predict(aval_final.reshape(1,1000))

In [66]:
print(previsao)

[1]


In [67]:
# Estrutura condicional para verificar o valor de previsao
if previsao == 1:
    print("O Texto Indica Sentimento Positivo!")
else:
    print("O Texto Indica Sentimento Negativo!")

O Texto Indica Sentimento Positivo!
